In [ ]:
import itertools
from collections import defaultdict
from tqdm import tqdm
import _pickle as cPickle
import os

# Dumping Paths
que_split_path_dict = {
    "train": ("../../datasets/VQA/cc-re/cc_v2_OpenEnded_mscoco_train2014_questions_88_split_{}.pkl",
              "../../datasets/VQA/v2_OpenEnded_mscoco_train2014_questions.json",
              "../../datasets/VQA/cc-re/train_cc_embeddings_88_split_{}.npy"),

    "val": ("../../datasets/VQA/cc-re/cc_v2_OpenEnded_mscoco_val2014_questions_88_split_{}.pkl",
            "../../datasets/VQA/v2_OpenEnded_mscoco_val2014_questions.json",
            "../../datasets/VQA/cc-re/val_cc_embeddings_88_split_{}.npy"),

}


ans_split_path_dict = {
    "train": ("../../datasets/VQA/cc-re/cc_train_target_88_split_{}.pkl",
              "../../datasets/VQA/cache/train_target.pkl"),
    "val": ("../../datasets/VQA/cc-re/cc_val_target_88_split_{}.pkl",
              "../../datasets/VQA/cache/val_target.pkl"),
}


ans2label_path = os.path.join("../../datasets/VQA/", "cache", "trainval_ans2label.pkl")
label2ans_path = os.path.join("../../datasets/VQA/", "cache", "trainval_label2ans.pkl")
ans2label = cPickle.load(open(ans2label_path, "rb"))
label2ans = cPickle.load(open(label2ans_path, "rb"))
answer_dict = {}
# use later
answers_data = []
for key in ans_split_path_dict.keys():
    answers = []
    path_holder = ans_split_path_dict[key][0]
    qpath_holder = que_split_path_dict[key][0]

    for i in range(4):
        path = path_holder.format(i)
        qpath = qpath_holder.format(i)
        
        split_answers = cPickle.load(open(path, "rb"))
        split_questions = cPickle.load(open(qpath, "rb"))
        
        for alist, qlist in zip(split_answers, split_questions):
            for a,q in zip(alist, qlist):
                a["question_id"] = q["question_id"]
        
        cPickle.dump(split_answers, open(path, "wb"))
        answers.extend(split_answers)
    answers_data.append(answers)
    answers = list(itertools.chain.from_iterable(answers))
    for ans in answers:
        answer_dict[ans["question_id"]] = ans["labels"]
print("Read Answers")

negs_path = "../../datasets/VQA/cc-re/trainval_question_negs_88_v2.pkl"
negs_data = cPickle.load(open(negs_path, "rb"))
negs_dict = {}
for qid, sim_scores, sim_qids in zip(negs_data["qids"], negs_data["sim_scores"], negs_data["sim_qids"]):
    negs_dict[qid] = (sim_scores, sim_qids)
print("Read Negatives Matrix")

# create dicts
image_dict = defaultdict(list)
questions_rephrasings = defaultdict(list)
question_dict = {}

# use later
questions_data = []
qids_dict = {}
for key in que_split_path_dict.keys():
    path_holder = que_split_path_dict[key][0]
    questions_list = []
    for i in range(4):
        path = path_holder.format(i)
        split_questions = cPickle.load(open(path, "rb"))
        questions_list.extend(split_questions)

    questions_data.append(questions_list)

    _dict = {}
    # add "rephrasing_of" key
    for _questions in questions_list:
        # only keep the min-qid in same-image ids
        min_qid = min([x['question_id'] for x in _questions])
        assert len(set([x['image_id'] for x in _questions])) == 1
        image_dict[_questions[0]["image_id"]].append(min_qid)
        for _que in _questions:
            question_dict[_que["question_id"]] = _que["question"]
            _dict[_que["question_id"]] = None
    qids_dict[key] = _dict
print("Read Questions")



In [ ]:
save_negs_dict_train = {}
save_negs_dict_val = {}
save_negs_dict_trainval = {}

def filter_negatives(sample):
    # filter same-image questions
    save_dict = {}
    same_image_ids = image_dict[sample["image_id"]]
    fil_same_image_ids = []
    ref_answers = answer_dict[sample["question_id"]]
    for qid in same_image_ids:
        if qid == sample["question_id"]:
            continue
        cand_answers = answer_dict[qid]
        if len(set(ref_answers).intersection(set(cand_answers))) == 0:
            fil_same_image_ids.append(qid)
    save_dict["same_image_questions_neg"] = fil_same_image_ids

    # filter top-k questions
    if sample["question_id"] not in negs_dict:
        return True

    top_k_sim_scores, top_k_questions = negs_dict[sample["question_id"]]
    fil_top_k_questions = []
    fil_top_k_questions_train = []
    fil_top_k_questions_val = []
    
    for qid in top_k_questions:
        cand_answers = answer_dict[qid]
        if len(set(ref_answers).intersection(set(cand_answers))) == 0:
            fil_top_k_questions.append(qid)
            if qid in qids_dict["train"]:
                fil_top_k_questions_train.append(qid)
            elif qid in qids_dict["val"]:
                fil_top_k_questions_val.append(qid)
            else:
                import pdb
                pdb.set_trace()
    
    save_dict["top_k_questions_neg"] = fil_top_k_questions
    save_dict["top_k_questions_neg_train"] = fil_top_k_questions_train
    save_dict["top_k_questions_neg_val"] = fil_top_k_questions_val

    if sample["question_id"] in qids_dict["train"]:
        del save_dict["top_k_questions_neg_val"]
        save_negs_dict_trainval[sample["question_id"]] = save_dict
        save_negs_dict_train[sample["question_id"]] = save_dict
    elif sample["question_id"] in qids_dict["val"]:
        del save_dict["top_k_questions_neg_train"]
        save_negs_dict_trainval[sample["question_id"]] = save_dict
        save_negs_dict_val[sample["question_id"]] = save_dict
    else:
        import pdb
        pdb.set_trace() 
    return False

In [ ]:
for que_data, ans_data, que_path, ans_path in zip(questions_data, 
                                                  answers_data, 
                                                  que_split_path_dict.values(), 
                                                  ans_split_path_dict.values()):
    questions_list, answers, que_path, ans_path = que_data,\
                                        ans_data,\
                                        que_path[0],\
                                        ans_path[0]
    
    print("Questions Path: ", que_path)
    print("Answers Path: ", ans_path)
    # add "rephrasing_of" key
    for _questions in questions_list:
        rep_id = min([s['question_id'] for s in _questions])
        for _que in _questions:
            _que["rephrasing_of"] = rep_id
    
    assert len(questions_list) == len(answers)

    # remove questions w/o negatives
    for idx in tqdm(range(len(questions_list)), total=len(questions_list)):
        _updated_ques = []
        _updated_answers = []
        _questions = questions_list[idx]
        _answers = answers[idx]

        for _que, _ans in zip(_questions, _answers):
            delete = filter_negatives(_que)
            if not delete:
                _updated_ques.append(_que)
                _updated_answers.append(_ans)
            else:
                import pdb
                pdb.set_trace()
        questions_list[idx] = _updated_ques
        answers[idx] = _updated_answers

# This was a hacky filtering procedure for removing unused samples, not needed any further.
#     cPickle.dump(data, open(que_path, "wb"), protocol=2)
#     cPickle.dump(answers, open(ans_path, "wb"), protocol=2)
#     print(f"Dumped: {que_path}")
#     print(f"Dumped: {ans_path}")

In [ ]:
save_negs_paths = {
   "val": "../../datasets/VQA/cc-re/cc_re_val_question_negs_v2.pkl",
   "train": "../../datasets/VQA/cc-re/cc_re_train_question_negs_v2.pkl",
   "trainval": "../../datasets/VQA/cc-re/cc_re_trainval_question_negs_v2.pkl",               
}

In [ ]:
values = list(save_negs_dict_val.values())
lens = [len(x["top_k_questions_neg"]) for x in values]
max(lens)

In [ ]:
import numpy as np

# not saved "trainval" file
for key in save_negs_paths.keys():
    negs_dict = globals()[f"save_negs_dict_{key}"]
    print(f"Save Path: {save_negs_paths[key]}")

    question_ids = list(negs_dict.keys())
    question_values = list(negs_dict.values())
    question_negs = np.zeros((len(question_ids), 300), dtype=np.int64) + -1
    same_image_questions_neg = np.zeros((len(question_ids), 300), dtype=np.int64) + -1
    
    for idx in tqdm(range(len(question_negs))):
        if key == "trainval":
            negs_ids = question_values[idx][f"top_k_questions_neg"]
        else:
            neg_ids = question_values[idx][f"top_k_questions_neg_{key}"]
        same_image_ids = question_values[idx]["same_image_questions_neg"]
        question_negs[idx][:len(neg_ids)] = neg_ids
        try:
            same_image_questions_neg[idx][:len(same_image_ids)] = same_image_ids
        except:
            import pdb
            pdb.set_trace()
    
    cPickle.dump({"qids": question_ids, 
                "question_negs": question_negs, 
                "same_image_questions_neg": same_image_questions_neg}, 
         open(save_negs_paths[key], "wb"), protocol=4)

In [ ]:
idx = 2
print(f"Original: {question_dict[question_ids[idx]]} \n -----------------")
nids = question_negs[idx]
for _id in nids:
    if _id > 0:
        print(_id)
        print(question_dict[_id])

In [ ]:
import _pickle as cPickle

save_negs_paths = {
   "qval": "../../datasets/VQA/cc-re/cc_v2_OpenEnded_mscoco_val2014_questions_88_split_{}.pkl",
   "qtrain": "../../datasets/VQA/cc-re/cc_v2_OpenEnded_mscoco_train2014_questions_88_split_{}.pkl",
   "aval": "../../datasets/VQA/cc-re/cc_val_target_88_split_{}.pkl",
   "atrain": "../../datasets/VQA/cc-re/cc_train_target_88_split_{}.pkl"
}

for split in ["train", "val"]:
    qsp, asp = f"cc_v2_OpenEnded_mscoco_{split}2014_questions_88_v2.pkl", \
            f"cc_{split}_target_88_v2.pkl"
    ql, al = [], []
    for i in range(4):
        qp, ap = save_negs_paths[f"q{split}"].format(i), save_negs_paths[f"a{split}"].format(i)
        qd, ad = cPickle.load(open(qp, "rb")), cPickle.load(open(ap, "rb"))
        ql.extend(qd)
        al.extend(ad)
    
    import pdb
    pdb.set_trace()

    cPickle.dump(ql, open(qsp, "wb"))
    cPickle.dump(al, open(asp, "wb"))
    


In [5]:
import os
import json
os.chdir("/nethome/ykant3/vilbert-multi-task/save/")
dirs = [
#     "VQA_spatial_m4c_mmt_vqa-final-ablate-scl-alt/",
#     "VQA_spatial_m4c_mmt_vqa-final-ablate-scl-alt-hard/",
#     "VQA_spatial_m4c_mmt_vqa-final-ablate-scl-alt-sc20/",
#     "VQA_spatial_m4c_mmt_vqa-final-ablate-scl-ce-batches/",
#     "VQA_spatial_m4c_mmt_vqa-final-ablate-scl-pretrain/",
#     "VQA_spatial_m4c_mmt_vqa-final-ablate-scl-scl-batches/",
#     "VQA_spatial_m4c_mmt_vqa-final-baseline/",
#     "VQA_spatial_m4c_mmt_vqa-final-baseline-train-minval-old/",
#     "VQA_spatial_m4c_mmt_vqa-final-baseline-trainval/",
#     "VQA_spatial_m4c_mmt_vqa-final-best/",
#     "VQA_spatial_m4c_mmt_vqa-final-best-0.7/",
#     "VQA_spatial_m4c_mmt_vqa-final-best-0.8/",
#     "VQA_spatial_m4c_mmt_vqa-final-best-0.9/",
#     "VQA_spatial_m4c_mmt_vqa-final-best-fourth/",
#     "VQA_spatial_m4c_mmt_vqa-final-best-img/",
#     "VQA_spatial_m4c_mmt_vqa-final-best-que/",
#     "VQA_spatial_m4c_mmt_vqa-final-best-second/",
#     "VQA_spatial_m4c_mmt_vqa-final-best-third/",
#     "VQA_spatial_m4c_mmt_vqa-final-best-trainval/",
#     "VQA_spatial_m4c_mmt_vqa-final-best-trainval-large/",
#     "VQA_spatial_m4c_mmt_vqa-final-best-trainval-long/",
#     "VQA_spatial_m4c_mmt_vqa-final-best-two-norm/",
#     "VQA_spatial_m4c_mmt_vqa-final-best-two-norm-second/",
#     "VQA_spatial_m4c_mmt_vqa-final-cc-baseline-allowed-18k/",
#     "VQA_spatial_m4c_mmt_vqa-final-cc-best-allowed-18k/",
#     "VQA_spatial_m4c_mmt_vqa-final-ce-scl-batches/",
#     "VQA_spatial_m4c_mmt_vqa-final-no-aug-baseline-train/",
#     "VQA_spatial_m4c_mmt_vqa-final-no-aug-baseline-trainval/",
#     "VQA_spatial_m4c_mmt_vqa-final-scl-fintune/",
#     "VQA_spatial_m4c_mmt_vqa-final-scl-fintune-10x/",
#     "VQA_spatial_m4c_mmt_vqa-final-scl-fintune-full/",
#     "VQA_spatial_m4c_mmt_vqa-final-scl-pretrain-fixed/",
#     "VQA_spatial_m4c_mmt_vqa-final-scl-fintune-full/",
#     "VQA_spatial_m4c_mmt_vqa-final-scl-fintune-10x/",
#     "VQA_spatial_m4c_mmt_vqa-final-cc-baseline-v2/",
#     "VQA_spatial_m4c_mmt_vqa-final-cc-best-v2/",
#     "VQA_spatial_m4c_mmt_vqa-final-scl-alt-sc20-bs-200-seed-10-no-eval-last/",
    "VQA_spatial_m4c_mmt_vqa-final-scl-fintune-full-v2/"
]

for _dir in dirs:
    preds_file = f"{_dir}preds_revqa_val.json"
    if os.path.exists(preds_file):
        preds = json.load(open(preds_file, "r"))["results"]
        val_scores = [x['vqa_score'] for x in list(preds.values())]
        print(f"Exp: {_dir}\n Val Score: {round(sum(val_scores)/len(val_scores), 4)} | Len: {len(val_scores)}")
        print("-----------")
        

Exp: VQA_spatial_m4c_mmt_vqa-final-scl-fintune-full-v2/
 Val Score: 0.6421 | Len: 214354
-----------


In [3]:
ls -1

archive-cc/
archive-cc-poor/
nohup.out
VQA_spatial_m4c_mmt_vqa-baseline-second/
VQA_spatial_m4c_mmt_vqa-baseline-third/
VQA_spatial_m4c_mmt_vqa-best-scl-alt-sc20-gpus-5-w12-seed-1000/
VQA_spatial_m4c_mmt_vqa-best-scl-pretrain-no-hard-no-scale/
VQA_spatial_m4c_mmt_vqa-best-unfil-ref-10/
VQA_spatial_m4c_mmt_vqa-best-unfil-ref-10-cf-2/
VQA_spatial_m4c_mmt_vqa-best-unfil-ref-10-cf-3/
VQA_spatial_m4c_mmt_vqa-best-unfil-ref-10-que-0.7-seed-10/
VQA_spatial_m4c_mmt_vqa-best-unfil-ref-10-que-0.8-seed-1000/
VQA_spatial_m4c_mmt_vqa-best-unfil-ref-10-resume/
VQA_spatial_m4c_mmt_vqa-best-unfil-ref-10-second/
VQA_spatial_m4c_mmt_vqa-best-unfil-ref-10-trainval/
VQA_spatial_m4c_mmt_vqa-best-unfil-ref-10-val-100/
VQA_spatial_m4c_mmt_vqa-best-unfil-ref-40-bs-400-eval-bt/
VQA_spatial_m4c_mmt_vqa-best-unfil-ref-50/
VQA_spatial_m4c_mmt_vqa-cc-baseline/
VQA_spatial_m4c_mmt_vqa-cc-baseline-allowed/
VQA_spatial_m4c_mmt_vqa-cc-baseline-eval-cc/
VQA_spatial_m4c_mmt_vqa-cc-best/
VQA_spatial_m4c_mmt_vqa-cc-best-a